In [1]:
api_Key = 'nHkOmbo3ziv8ugEwGEk5K1eGv41m8NatuB43abFD'

In [3]:
!pip install requests

In [5]:
import requests
from datetime import datetime

api_key = "nHkOmbo3ziv8ugEwGEk5K1eGv41m8NatuB43abFD"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={api_key}"
target = 10000
asteroids_data = []

while len(asteroids_data) < target and url:
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}, stopping.")
        break
    data = response.json()
    near_earth_objects = data.get('near_earth_objects', {})

    for date, asteroids in near_earth_objects.items():
        for asteroid in asteroids:
            close_approach_data = asteroid.get('close_approach_data', [])
            if not close_approach_data:
                continue
            close_approach = close_approach_data[0]
            try:
                asteroids_data.append({
                    'id': asteroid.get('id'),
                    'neo_reference_id': asteroid.get('neo_reference_id'),
                    'name': asteroid.get('name'),
                    'absolute_magnitude_h': float(asteroid.get('absolute_magnitude_h', 0.0)),
                    'estimated_diameter_min_km': float(asteroid.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_min', 0.0)),
                    'estimated_diameter_max_km': float(asteroid.get('estimated_diameter', {}).get('kilometers', {}).get('estimated_diameter_max', 0.0)),
                    'is_potentially_hazardous_asteroid': bool(asteroid.get('is_potentially_hazardous_asteroid', False)),
                    'close_approach_date': datetime.strptime(close_approach.get('close_approach_date'), "%Y-%m-%d").date() 
                    if close_approach.get('close_approach_date') else None,
                    'relative_velocity_kmph': float(close_approach.get('relative_velocity', {}).get('kilometers_per_hour', 0.0)),
                    'miss_distance_km': float(close_approach.get('miss_distance', {}).get('kilometers', 0.0)),
                    'astronomical': float(close_approach.get('miss_distance', {}).get('astronomical', 0.0)),
                    'miss_distance_lunar': float(close_approach.get('miss_distance', {}).get('lunar', 0.0)),
                    'orbiting_body': close_approach.get('orbiting_body', 'N/A')
                })
            except (ValueError, TypeError):
                continue
            if len(asteroids_data) >= target:
                break
        if len(asteroids_data) >= target:
            break
    url = data.get('links', {}).get('next')

In [7]:
len(asteroids_data)

10000

In [9]:
asteroids_data[0]

{'id': '2415949',
 'neo_reference_id': '2415949',
 'name': '415949 (2001 XY10)',
 'absolute_magnitude_h': 19.37,
 'estimated_diameter_min_km': 0.3552670883,
 'estimated_diameter_max_km': 0.7944013596,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': datetime.date(2024, 1, 2),
 'relative_velocity_kmph': 57205.8951204341,
 'miss_distance_km': 50452409.349026635,
 'astronomical': 0.3372535274,
 'miss_distance_lunar': 131.1916221586,
 'orbiting_body': 'Earth'}

In [11]:
import pymysql
myconnection = pymysql.connect(host='127.0.01', user='root',passwd='Sruthi@666')
cur=myconnection.cursor()
cur.execute("create database Near_Earth_Objects")

1

In [13]:
cur.execute("use Near_Earth_Objects")

0

In [15]:
cur.execute("""
CREATE TABLE asteroids (
    id int PRIMARY KEY, 
    name varchar(40), 
    absolute_magnitude_h float, 
    estimated_diameter_min_km float, 
    estimated_diameter_max_km float, 
    is_potentially_hazardous_asteroid boolean)
""")
cur.execute("""
CREATE TABLE close_approach (
    neo_reference_id int, 
    close_approach_date date, 
    relative_velocity_kmph float, 
    astronomical float, 
    miss_distance_km float, 
    miss_distance_lunar float, 
    orbiting_body varchar(20),
    FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id))
""")

0

In [17]:
myconnection.commit()

In [19]:
for ast in asteroids_data:
    cur.execute("""
    INSERT IGNORE INTO asteroids (
        id, name, absolute_magnitude_h, 
        estimated_diameter_min_km, estimated_diameter_max_km, 
        is_potentially_hazardous_asteroid) VALUES (%s, %s, %s, %s, %s, %s)
""", (
    ast['id'],
    ast['name'],
    ast['absolute_magnitude_h'],
    ast['estimated_diameter_min_km'],
    ast['estimated_diameter_max_km'],
    ast['is_potentially_hazardous_asteroid']
))
myconnection.commit()

In [21]:
for ast in asteroids_data:
    cur.execute("SELECT 1 FROM asteroids WHERE id = %s", (ast['neo_reference_id'],))
    if cur.fetchone():  
        cur.execute("""
            INSERT INTO close_approach (
                neo_reference_id, close_approach_date, relative_velocity_kmph,
                astronomical, miss_distance_km, miss_distance_lunar, orbiting_body
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            ast['neo_reference_id'],
            ast['close_approach_date'],
            ast['relative_velocity_kmph'],
            ast['astronomical'],
            ast['miss_distance_km'],
            ast['miss_distance_lunar'],
            ast['orbiting_body']
        ))
myconnection.commit()